# Parsear y procesar la salida

In [2]:
from langchain.prompts import PromptTemplate,SystemMessagePromptTemplate,ChatPromptTemplate,HumanMessagePromptTemplate
from langchain_openai import ChatOpenAI
f=open('openai_api_key.txt')
api_key=f.read()
chat=ChatOpenAI(openai_api_key=api_key)

## Parsear uns lista de elemento separados por coma

In [5]:
from langchain.output_parsers import CommaSeparatedListOutputParser

In [6]:
output_parser=CommaSeparatedListOutputParser()

In [7]:
format_instruction=output_parser.get_format_instructions()

In [8]:
print(format_instruction)

Your response should be a list of comma separated values, eg: `foo, bar, baz` or `foo,bar,baz`


In [9]:
respuesta="coche, arbol, carretera"
output_parser.parse(respuesta)

['coche', 'arbol', 'carretera']

In [10]:
human_template='{request}\n{format_instructions}'
human_prompt=HumanMessagePromptTemplate.from_template(human_template)

In [12]:
chat_prompt=ChatPromptTemplate.from_messages([human_prompt])

chat_prompt.format_prompt(
    request="dime 5 caracteristicas de los coches americanos",
    format_instructions=output_parser.get_format_instructions()
)

ChatPromptValue(messages=[HumanMessage(content='dime 5 caracteristicas de los coches americanos\nYour response should be a list of comma separated values, eg: `foo, bar, baz` or `foo,bar,baz`', additional_kwargs={}, response_metadata={})])

In [17]:
solicitud_completa=chat_prompt.format_prompt(
    request="dime 5 caracteristicas de la musica ecuatoriana",
    format_instructions=output_parser.get_format_instructions()
).to_messages()

In [18]:
result=chat.invoke(solicitud_completa)

In [19]:
result.content

'ritmo alegre, melodía pegajosa, letras emotivas, instrumentos andinos, fusiones de géneros musicales.'

In [20]:
output_parser.parse(result.content)

['ritmo alegre',
 'melodía pegajosa',
 'letras emotivas',
 'instrumentos andinos',
 'fusiones de géneros musicales.']

# Parsear formatos de fecha

In [21]:
from langchain.output_parsers import DatetimeOutputParser

In [22]:
output_parser=DatetimeOutputParser()

In [23]:
print(output_parser.get_format_instructions())

Write a datetime string that matches the following pattern: '%Y-%m-%dT%H:%M:%S.%fZ'.

Examples: 1570-04-09T17:25:47.499046Z, 1277-07-28T21:09:12.098076Z, 1387-06-20T05:42:54.268959Z

Return ONLY this string, no other words!


In [24]:
template_text="{request}\n{format_instructions}"
human_prompt=HumanMessagePromptTemplate.from_template(template_text)

In [25]:
chat_prompt=ChatPromptTemplate.from_messages([human_prompt])

In [27]:
solicitud_completa=chat_prompt.format_prompt(
    request="dime la fecha de muerte de simon bolivar.",
    format_instructions=output_parser.get_format_instructions()
).to_messages()

In [29]:
result=chat.invoke(solicitud_completa)

In [30]:
result.content

'1830-12-17T18:30:00.000000Z'

In [31]:
output_parser.parse(result.content)

datetime.datetime(1830, 12, 17, 18, 30)

# Metodos para solucionar problemas de parseo 

## Auto-fix parser
Va a reiterar la llamada al LLM para que solucione el mal parseo.

In [33]:
from langchain.output_parsers import OutputFixingParser
output_parser_dates=DatetimeOutputParser()
misformatted=result.content

In [34]:
misformatted

'1830-12-17T18:30:00.000000Z'

In [35]:
new_parser=OutputFixingParser.from_llm(
    parser=output_parser_dates,
    llm=chat
)

In [37]:
new_parser.parse(misformatted)

datetime.datetime(1830, 12, 17, 18, 30)

## Solucionar con SystemPrompt

In [47]:
system_prompt=SystemMessagePromptTemplate.from_template("Tienes que responder únicamente con un patrón de fechas.")
template_text="{request}\n{format_instructions}"
human_prompt=HumanMessagePromptTemplate.from_template(template_text)

In [48]:
chat_prompt=ChatPromptTemplate.from_messages([system_prompt,human_prompt])

In [49]:
print(
    chat_prompt.format(
        request="Cual es la fecha de independencia de Guayaquil",
        format_instructions=output_parser_dates.get_format_instructions()
    )
)

System: Tienes que responder únicamente con un patrón de fechas.
Human: Cual es la fecha de independencia de Guayaquil
Write a datetime string that matches the following pattern: '%Y-%m-%dT%H:%M:%S.%fZ'.

Examples: 0344-10-03T15:14:41.275316Z, 0116-10-13T22:31:36.122370Z, 0805-02-03T18:40:32.249313Z

Return ONLY this string, no other words!


In [51]:
solicitud_completa=chat_prompt.format_prompt(
        request="Cual es la fecha de independencia de Guayaquil",
        format_instructions=output_parser_dates.get_format_instructions()
    ).to_messages()

In [52]:
result=chat.invoke(solicitud_completa)

In [53]:
result.content

'1820-10-09T00:00:00.000000Z'

In [54]:
output_parser_dates.parse(result.content)

datetime.datetime(1820, 10, 9, 0, 0)